In [14]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold,RepeatedKFold, GridSearchCV,  RandomizedSearchCV
import math
from data import *
import pandas as pd
from itertools import islice
import seaborn as sns; sns.set_theme()
from pandas import HDFStore

In [15]:
hdf = HDFStore('/Users/lukemcdermott/Desktop/Physics/single_templates_flux_calibrated_version_july22.h5', mode='r')

In [17]:
hdf.keys()

['/singles_difference_spectrum', '/singles_flux', '/singles_noise']

In [19]:
singles_noise = pd.read_hdf('/Users/lukemcdermott/Desktop/Physics/single_templates_flux_calibrated_version_july22.h5', key = '/singles_noise')
singles_difference_spectrum = pd.read_hdf('/Users/lukemcdermott/Desktop/Physics/single_templates_flux_calibrated_version_july22.h5', key = '/singles_difference_spectrum')
singles_flux = pd.read_hdf('/Users/lukemcdermott/Desktop/Physics/single_templates_flux_calibrated_version_july22.h5', key = '/singles_flux')

In [20]:
display(singles_noise)

,flux_0,flux_1,flux_2,flux_3,flux_4,flux_5,flux_6,flux_7,flux_8,flux_9,...,flux_436,flux_437,flux_438,flux_439,flux_440,spectral_type,object_name,spex_type,opt_type,obs_date
0,1.845473e-12,1.923989e-12,1.934470e-12,1.989010e-12,2.021825e-12,2.021588e-12,2.024703e-12,1.999796e-12,1.964424e-12,1.923532e-12,...,1.095609e-12,1.060906e-12,1.098662e-12,1.042024e-12,1.072057e-12,M6.0,2MASS J16002535-2644060,M6.0,nan,20120608
1,1.202773e-11,1.227993e-11,1.190583e-11,1.219745e-11,1.177336e-11,1.150248e-11,8.946026e-12,9.626168e-12,1.034666e-11,1.055017e-11,...,3.301673e-12,3.241883e-12,3.232486e-12,3.201478e-12,3.167601e-12,d/sdM6.0,2MASS J17252029-0024508,M6.0,nan,20030523
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,d/sdM5.0,WISE J141143.25-452418.3,M6.0,nan,20140504
4,9.786525e-12,9.357863e-12,9.322127e-12,9.081116e-12,9.273860e-12,9.064975e-12,9.287678e-12,9.117092e-12,8.756505e-12,8.450147e-12,...,4.229760e-12,4.213895e-12,3.903356e-12,3.742201e-12,3.925670e-12,d/sdM6.0,2MASS J19445221-0831036,M6.0,nan,20030917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
display(singles_flux)

,flux_0,flux_1,flux_2,flux_3,flux_4,flux_5,flux_6,flux_7,flux_8,flux_9,...,flux_436,flux_437,flux_438,flux_439,flux_440,spectral_type,object_name,spex_type,opt_type,obs_date
0,1.681441e-10,1.924531e-10,2.087709e-10,2.249161e-10,2.335279e-10,2.357139e-10,2.388972e-10,2.392189e-10,2.335245e-10,2.383486e-10,...,6.881774e-11,6.805103e-11,6.975343e-11,6.805273e-11,6.880741e-11,M6.0,2MASS J16002535-2644060,M6.0,nan,20120608
1,2.222151e-10,2.302853e-10,2.503629e-10,2.594328e-10,2.464011e-10,2.457234e-10,2.536640e-10,2.449596e-10,2.525856e-10,2.582562e-10,...,7.211824e-11,7.038709e-11,7.294774e-11,7.565103e-11,7.872028e-11,d/sdM6.0,2MASS J17252029-0024508,M6.0,nan,20030523
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,d/sdM5.0,WISE J141143.25-452418.3,M6.0,nan,20140504
4,1.571633e-10,1.820034e-10,1.973238e-10,2.064186e-10,2.359755e-10,2.542756e-10,2.089522e-10,2.121430e-10,2.098517e-10,2.247026e-10,...,8.917762e-11,7.627971e-11,7.776538e-11,8.266983e-11,8.200777e-11,d/sdM6.0,2MASS J19445221-0831036,M6.0,nan,20030917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Take 2 spectrums, add them together, figure out what this looks like